# Imports & Database

In [2]:
import requests
from bs4 import BeautifulSoup
from html import unescape
from datetime import date
import pandas as pd
import logging
import os
from functions import post, parse, nuevos_avisos, guardar_avisos
import base64

In [3]:
palabras_clave = {'energía','energético'}

In [4]:
# Cargar la base de datos existente
database = pd.read_excel('database.xlsx')
database = database.drop(index=database.index)

# Logger

In [5]:
# Configurar logger
logging.basicConfig(
    filename='boletin_errors.log',
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Descarga pdfs

In [6]:
def descargar_pdf(link, output_dir="pdfs"):
    """
    Descarga el PDF desde un link del Boletín Oficial, solo si no existe.
    Maneja correctamente la URL de redirección para obtener el archivo PDF real.
    """

    # Extraer tipo, número y fecha desde el link original
    partes = link.split('/')
    seccion = partes[-3]
    id_aviso = partes[-2]
    fecha_publicacion = partes[-1].split('?')[0]

    # Construir URL que apunta al visor
    #url_pdf_viewer = f"https://www.boletinoficial.gob.ar/web/utils/pdfView?file=%2Fpdf%2Faviso%2F{seccion}%2F{id_aviso}%2F{fecfecha_publicacionha}"
    file_name = f"{id_aviso}_{fecha_publicacion}.pdf"


    # URL del endpoint
    url_pdf = "https://www.boletinoficial.gob.ar/avisos/pdf/getAvisoPdf"

    # Headers necesarios
    headers = {
        "Referer": f"https://www.boletinoficial.gob.ar/detalleAviso/{seccion}/{id_aviso}/{fecha_publicacion}",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest"
    }

    # Payload de la solicitud
    data = {
        "nombreSeccion": seccion,
        "idAviso": id_aviso,
        "fechaPublicacion": fecha_publicacion
    }

    # Realizar el POST
    response = requests.post(url_pdf, headers=headers, data=data)

    # Revisar la respuesta
    if response.status_code == 200:
        json_data = response.json()
        if 'pdfBase64' in json_data and json_data['pdfBase64']:
            pdf_base64 = json_data['pdfBase64']
            pdf_bytes = base64.b64decode(pdf_base64)

            with open(file_name, "wb") as f:
                f.write(pdf_bytes)

            print(f"PDF guardado como {file_name}")
        else:
            print("⚠️ No se encontró el PDF en la respuesta.")
    else:
        print(f"❌ Error HTTP {response.status_code}")
    # try:
    #     os.makedirs(output_dir, exist_ok=True)

    #     partes = link_aviso.split("/detalleAviso/")[1].split("?")[0].split("/")
    #     tipo, numero, fecha = partes

    #     pdf_url = f"https://www.boletinoficial.gob.ar/web/utils/pdfView?file=%2Fpdf%2Faviso%2F{tipo}%2F{numero}%2F{fecha}"
    #     nombre_archivo = f"{numero}_{fecha}.pdf"
    #     ruta_completa = os.path.join(output_dir, nombre_archivo)

    #     response = requests.get(pdf_url)
    #     print (response.status_code)
    #     if response.status_code == 200:
    #         with open(ruta_completa, "wb") as f:
    #             f.write(response.content)
    #         logging.info(f"PDF descargado: {nombre_archivo}")
    #         return ruta_completa
    #     else:
    #         logging.warning(f"No se pudo descargar el PDF: {pdf_url}")
    #         return None
    # except Exception as e:
    #     logging.error(f"Error al descargar PDF de {link_aviso}: {e}")
    #     return None

In [7]:
avisos_totales = []
for palabra in palabras_clave :
    try:
        response = post(palabra)
        if response.status_code == 200:
            datos = response.json()
            avisos_totales.extend(parse(datos))
        else:
            logging.warning(f"Error al buscar '{palabra}': Código {response.status_code}")
    except Exception as e:
        logging.error(f"Fallo la búsqueda para '{palabra}': {e}")

In [8]:
if avisos_totales is not None:
    df_nuevos_avisos = nuevos_avisos(avisos_totales, database)

In [9]:
if not df_nuevos_avisos.empty:
    guardar_avisos(df_nuevos_avisos,database)
    for index, row in df_nuevos_avisos.iterrows():
        link_aviso = row['Link']
        #descargar_pdf(link_aviso)

In [11]:
df_nuevos_avisos

,Resolución,Ministerio,Link,Fecha de Publicación,Referencia,Contenido,Resumen
0,Resolución 223/2025,SECRETARÍA DEENERGÍA,https://www.boletinoficial.gob.ar/detalleAviso...,Fecha de Publicacion: 28/05/2025,RESOL-2025-223-APN-SE#MEC,la Resolución Nº 61 de fecha 29 de abril de 19...,None
1,Resolución 224/2025,SECRETARÍA DEENERGÍA,https://www.boletinoficial.gob.ar/detalleAviso...,Fecha de Publicacion: 28/05/2025,RESOL-2025-224-APN-SE#MEC,la Resolución Nº 61 de fecha 29 de abril de 19...,None
